In [2]:
#
from sklearn import metrics
import pandas as pd
import numpy as np
from scipy.sparse import *
from sklearn.metrics.pairwise import pairwise_distances
import math
import random
import matplotlib.pyplot as plt
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier 
import time
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
res = pd.DataFrame(0,columns=["spnsi","speci","GM","f1","AUC"],index=dataList)


In [38]:
dataList = ["MM/MM-BP.csv","MM/MM-CC.csv","MM/MM-MF.csv","MM/MM-BPCC.csv",
            "MM/MM-BPMF.csv","MM/MM-CCMF.csv","MM/MM-BPCCMF.csv",
           "CE/CE-BP.csv","CE/CE-CC.csv","CE/CE-MF.csv","CE/CE-BPCC.csv",
            "CE/CE-BPMF.csv","CE/CE-CCMF.csv","CE/CE-BPCCMF.csv",
           "DM/DM-BP.csv","DM/DM-CC.csv","DM/DM-MF.csv","DM/DM-BPCC.csv",
            "DM/DM-BPMF.csv","DM/DM-CCMF.csv","DM/DM-BPCCMF.csv",
           "SC/SC-BP.csv","SC/SC-CC.csv","SC/SC-MF.csv","SC/SC-BPCC.csv",
            "SC/SC-BPMF.csv","SC/SC-CCMF.csv","SC/SC-BPCCMF.csv"]
dataList = ["SportsTweetst.csv","SportsTweetsc.csv","NYDailyheadings.csv","Stumbleupon.csv","Cities.csv"]
dataName = "Stumbleupon.csv"

In [39]:
#加载数据 预处理
AncDict = {}
DecDict = {}
relativeList = []
sparseList = []
threshold = 0.999
F1 = []
AUC = []
featureNum = []
pathList = []
numList = []
sensitivity = []
specificity = []
data = pd.read_csv("../data/real_world_datasets/Datasets/"+dataName,index_col=None,encoding='gbk')
if(dataName == "Cities.csv"):
    data["longevity influence"] = data["longevity influence"].apply(lambda x : 0 if x < 2e+09 else 1)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

#初始化基因本体的DAG
DAG = pd.read_csv("../data/real_world_datasets/Hierarchy/"+dataName,index_col=0)

#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 30):
        data.drop(c,axis=1,inplace=True)
interColumn = DAG.columns.intersection(data.columns)
DAG = DAG.loc[interColumn,interColumn]
data = data.loc[:,interColumn]
print(data.shape)
#计算相关性
IGWeight = mutual_info_classif(data,label,discrete_features=True)
IGWeight = pd.Series(IGWeight,index=data.columns)

#计算相关性
data["longevity influence"] = label
Relevance1 = pd.Series(0.0,index=DAG.columns)
for name in data.columns:
    temp = data[data[name] == 1]
    prob = temp['longevity influence'].mean()
    Relevance1[name] = (prob-0.5)*(prob-0.5)+(0.5-prob)*(0.5-prob)
    
Relevance0 = pd.Series(0.0,index=data.columns)
for name in data.columns:
    temp = data[data[name] == 0]
    prob = temp['longevity influence'].mean()
    Relevance0[name] = (prob-0.5)*(prob-0.5)+(0.5-prob)*(0.5-prob)  
data = data.drop(["longevity influence"],axis=1)

#计算所有节点的祖先节点
for name in data.columns:
    global relativeList
    relativeList = []
    getAncestors(name)
    AncDict[name] = relativeList
#计算所有节点的孩子节点
for name in data.columns:
    global relativeList
    relativeList = []
    getDecestors(name)
    DecDict[name] = relativeList


(3020, 431)


In [3]:
#函数
def Eager(trainData):
    global pathList
    leafNodes = []
    returnList = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    for leaf in leafNodes:
        pathList = []
        getPathsToRoot([],leaf)
        for path in pathList:
            
            #path = list(filter(lambda x:x in DAG.columns,path))
            pathIG = IGWeight[path]
            highIG = pathIG[pathIG >= pathIG.mean()]
            for index in highIG.index:
                returnList.append(index)
    
    return list(set(returnList))

def Lazy(x):
    #step1
    dropList = []
    for name in DAG.columns:
        if(x[name] == 1):
            #遍历当前节点的所有祖先节点
            for anc in AncDict[name]:
                if(Relevance1[anc] < Relevance1[name]):
                    dropList.append(anc)
        else:
            dropList.append(name)
    return list(set(DAG.columns)-set(dropList))

def test(x,trainData):
    global pathList
    #找到所有叶子结点
    leafNodes = []
    dropList = []
    returnList = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    #step1
    dropList = []
    for name in DAG.columns:
        if(x[name] == 1):
            #遍历当前节点的所有祖先节点
            for anc in AncDict[name]:
                if(Relevance1[anc] < Relevance1[name]):
                    dropList.append(anc)
        else:
            dropList.append(name)
    selectFeatures = list(set(DAG.columns)-set(dropList))

    print(len(selectFeatures))
    #step2
    leafNodes = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    for leaf in leafNodes:
        pathList = []
        getPathsToRoot([],leaf)
        for path in pathList:
            path = list(filter(lambda x:x in selectFeatures,path))
            pathIG = IGWeight[path]
            highIG = pathIG[pathIG >= pathIG.mean()]
            for index in highIG.index:
                returnList.append(index)
    return list(set(returnList))

def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
        
def RPV(x):
    dropList = []
    for name in DAG.columns:
        if(x[name] == 1):
            #遍历当前节点的所有祖先节点
            for anc in AncDict[name]:
                if(Relevance1[anc] < Relevance1[name]):
                    dropList.append(anc)
        else:
            dropList.append(name)
    return list(set(DAG.columns)-set(dropList))

def MR(x):
    global pathList
    dropSet = set()
    for name in DAG.columns:
        pathList = []
        if(x[name] == 1):#找到所有至根节点路径
            getPathsToRoot([],name)
            for path in pathList:
                #保留相关性最高特征，其它特征加入dropSet
                dropSet.update(Relevance1[path].sort_values(ascending=False).index[1:])
        else:
            dropSet.add(name)
#             getPathsToLeft([],name)
#             for path in pathList:
#                 #保留相关性最高特征，其它特征加入dropSet
#                 dropSet.update(Relevance0[path].sort_values(ascending=False).index[1:])
        #print(pathList)
    return list(set(DAG.columns)-dropSet)

def myknn(train,label,x_test):
    predictList = []
    for j in range(x_test.shape[0]):
        test = x_test.iloc[j,:]
        maxJcd = 0
        prediction = 0
        for i in range(train.shape[0]):
            temp = jaccard_score(train.iloc[i,:],test)
            if(temp > maxJcd):
                prediction = label[i]
                maxJcd = temp
        predictList.append(prediction)
    return predictList

def getAncestors(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[relative,name] == 1):
            relativeList.append(relative)
            getAncestors(relative)
def getDecestors(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[name,relative] == 1):
            relativeList.append(relative)
            getDecestors(relative)
def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
def getPathsToLeft(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for dec in DAG.columns:
        if(DAG.loc[name,dec] == 1):
            flag = 1
            getPathsToLeft(copy.copy(path),dec)
    if(flag == 0):
        pathList.append(path)

In [41]:
#10折交叉验证
kf = KFold(n_splits=5,shuffle=True)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证
    predicetList = []
    
    feat1 = Eager(trainData)
    for i in range(testData.shape[0]):  #对每一个样本进行特征选择和预测
        
        feat2 = Lazy(testData.iloc[i,:]) #RPV特征选择
        #selectFeatures = list(set(feat1).union(set(feat2))) #并集
        selectFeatures = list(set(feat1).intersection(set(feat2)))
        
        print(len(feat1),len(feat2),len(selectFeatures))
        
        if(len(selectFeatures) == 0):
            predicetList.append(pd.Series(Y_train).mode()[0])
            continue
            #selectFeatures = DAG.columns
        else:
            numList.append(len(selectFeatures))
        X_train = trainData.loc[:,selectFeatures]
        X_test = testData.loc[:,selectFeatures].iloc[i,:]
        from sklearn.svm import SVC
        clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
        clf.fit(X_train, Y_train)
        predicetList.append(clf.fit(X_train, Y_train).predict(X_test.values.reshape(1,len(selectFeatures)))[0])
        
        
#         gnb = GaussianNB()
#         predicetList.append(gnb.fit(X_train, Y_train).predict(X_test.values.reshape(1,len(selectFeatures)))[0])#把当前样本预测结果加入List
#         #knn=KNeighborsClassifier()
        #knn.fit(X_train,Y_train)
        #predicetList.append(knn.predict(X_test.values.reshape(1,len(selectFeatures)))[0])
    print(predicetList)
    print(Y_test)
    sensi,speci= GM_score(np.array(predicetList),Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predicetList),Y_test))
    try:
        AUC.append(roc_auc_score(np.array(predicetList),Y_test))
    except ValueError:
        pass

#新方法
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(round(math.sqrt(a*b)*100,1))
print("f1")
print(round(np.nanmean(F1)*100,1))
print("AUC")
print(round(np.nanmean(AUC)*100,1))
print("选择特征百分比")
print(round(np.nanmean(numList)/DAG.shape[0]*100,1))
res.loc[dataName,"spnsi"] = round(a*100,1)
res.loc[dataName,"speci"] = round(b*100,1)
res.loc[dataName,"GM"] = round(math.sqrt(a*b)*100,1)
res.loc[dataName,"f1"] = round(np.nanmean(F1)*100,1)
res.loc[dataName,"AUC"] = round(np.nanmean(AUC)*100,1)

135 25 15


c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


135 3 2


c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


135 9 4


...

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


135 18 8


c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


135 6 4
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [37]:
res

,spnsi,speci,GM,f1,AUC
SportsTweetst.csv,68.0,70.7,69.3,59.4,69.4
SportsTweetsc.csv,85.0,93.1,89.0,85.4,89.1
NYDailyheadings.csv,46.1,60.3,52.8,55.0,53.2
Stumbleupon.csv,0.0,0.0,0.0,0.0,0.0
Cities.csv,54.0,55.5,54.7,45.2,54.8
